Creado por Ivan Gabriel corredor castillo \
30-05-2024

In [2]:
import pandas as pd
import numpy as np
import time

In [3]:
#instalar themas
#pip install jupyterlab-night
#https://github.com/topics/jupyterlab-theme
#https://jupyter-notebook.readthedocs.io/en/latest/migrating/custom-themes.html
#!jupyter --version

## Consolidar base geih por año

In [5]:
# rutas
## ruta entrada
p1 = "E:/GEIH/2022/" 
## ruta salida
p2 = "E:/GEIH/salidas/"

In [6]:
pd.options.display.max_columns = None

In [14]:
# medir tiempo de ejecución
#tiempo ejecición
start_time = time.time()
# libreria para pegar dataframes en serie
from functools import reduce

# 1. Crear listas tablas de entrada, salida y variables

# tablas de personas, 80115 registros
tab_in = ["Características generales, seguridad social en salud y educación",
             "Fuerza de trabajo", "Ocupados", "No ocupados", "Datos del hogar y la vivienda"]
# tablas de salida
meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio",
         "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
# separadores
sep = [",", ";", ";", ";", ";", ";",
       ";", ";", ";", ";", ";", ";"]

# variables de pegue
key_columns = ['DIRECTORIO', 'SECUENCIA_P', 'ORDEN']

#lista vacia para almacenar cada dfs por mes
frames_mes = []
for h, k in zip(meses, sep):
    # 2. cargue de las bases
    for i, j in zip(range(1,6), tab_in):
        df1 = pd.read_csv(p1 + f"{h}/CSV/{j}.csv", 
                          sep= k, decimal= ",", encoding="latin1", low_memory=False)
        df1.columns= df1.columns.str.upper()    
        exec(f"{h}_{i} = df1.copy()")
        del df1

    # 3. Get it non key common variables in both dataframes
    common_columns = list(set(eval(f"{h}_1").columns).intersection(set(eval(f"{h}_2").columns)).
                          intersection(set(eval(f"{h}_3").columns)).
                          intersection(set(eval(f"{h}_4").columns)).
                          intersection(set(eval(f"{h}_5").columns))- set(key_columns))

    # 4. Drop common columns from dataframes except características generales, before merging
    for i in range(2,6):
        eval(f"{h}_{i}").drop(columns=common_columns, inplace=True)

    # List of DataFrames
    frames_s = [eval(f"{h}_1"), eval(f"{h}_2"), eval(f"{h}_3"), eval(f"{h}_4")]
    # 5. Pegue modulos personas
    df2 = reduce(lambda left, right: pd.merge(left, right, on= key_columns, 
                                              how="left", 
                                              validate="1:1", #suffixes=("_l", "_R")
                                                ), frames_s)
    exec(f"{h}_per = df2.copy()")
    #borrar elemntos de cache
    del df2
    # 6. merge personas -hogares
    df3 =  pd.merge(left= eval(f"{h}_per"), right = eval(f"{h}_5"),
                            left_on=['DIRECTORIO',  'SECUENCIA_P'], 
                            right_on=['DIRECTORIO', 'SECUENCIA_P'],
                            how='left', sort=False, validate='m:1',
                            #suffixes=('_P', '_H'), 
                            indicator = 'INDICATOR_COLUMN')
    exec(f"{h}_22 = df3.copy()") 
    # borrar elementos de cache
    for i in range(1, 6):
        del globals()[f"{h}_{i}"] 
    del df3, globals()[f"{h}_per"]

    # 7. llenar la lista con los dfs por meses
    frames_mes.append(eval(f"{h}_22"))
# 8. Concatenar los meses
geih_22 = pd.concat(frames_mes, axis=0, ignore_index=True)
# tiempo final
end_time = time.time()
execution_time = end_time - start_time
print("tiempo de ejecución :", np.round(execution_time), "segundos\n")

tiempo de ejecución : 62.0 segundos



In [100]:
geih_22["MES"].value_counts()

MES
1     80116
2     79783
3     78595
6     77999
4     77679
5     77380
8     76625
7     75963
9     75382
11    73614
10    73477
12    72846
Name: count, dtype: int64

In [20]:
#prueba para el trimestre
print("población nacional:", (pd.to_numeric(Enero_22['FEX_C18']).sum() + 
 pd.to_numeric(Febrero_22['FEX_C18']).sum() + 
 pd.to_numeric(Marzo_22['FEX_C18']).sum())/3 )
#pd.to_numeric(Enero_22[Enero_22['P6240']==1]['FEX_C18']).mean()

# prueba población campesina
print("población campesina:", 
((pd.to_numeric(Enero_22[(Enero_22["P2057"]==1) & (Enero_22["P2061"]==1)]['FEX_C18']).sum()) +
(pd.to_numeric(Febrero_22[(Febrero_22["P2057"]==1) & (Febrero_22["P2061"]==1)]['FEX_C18']).sum()) +
(pd.to_numeric(Marzo_22[(Marzo_22["P2057"]==1) & (Marzo_22["P2061"]==1)]['FEX_C18']).sum()) 
)/ 3 )

población nacional: 50297750.33333716
población campesina: 14460983.610993758


In [25]:
# Pruebas para corroborar pegue de modulos
pd.to_numeric(Enero_22[Enero_22['P6240']==1]['FEX_C18']).mean()
#pd.to_numeric(Enero_2['FEX_C18']).mean()
# mean: mod_2['P6240']: 640.6648305254095, 
# mean: personas['P6240']: 640.6648305254095

640.6648305254095

## Base víctimas con geih

In [24]:
#rutas
p1 = "C:/Users/ivan.corredor/pobreza_calculos_2019_2022/2023/pobreza monetaria/"

In [46]:
# Cargue de tabla 2022
vic_22 = pd.read_stata(p1 + "Victimas_GEIH_2022.dta")
# convertir a mayúsculas los nombres de las columnas
vic_22.columns= vic_22.columns.str.upper()
#convertir variable mes a numero para pegue
vic_22["MES"] = pd.to_numeric(vic_22["MES"], errors='coerce')

In [48]:
# Merge geih con víctimas
geih_vic_22 = pd.merge(left = geih_22, 
                          right= vic_22, 
                          left_on=["DIRECTORIO", "SECUENCIA_P", "ORDEN", "MES"], 
                          right_on=["DIRECTORIO", "SECUENCIA_P", "ORDEN", "MES"], 
                          how='left', sort=False, validate='1:1')
                          #, suffixes=("_ph", "_v"), indicator = 'indicator_column_vhp'))

In [52]:
# crear variable de no víctima
def novic(df):
    if df["ES_VICTIMA"]!=1:
        x = 1
    else:
        x= 0
    return(x)
geih_vic_22["NO_VICTIMA"] = geih_vic_22.apply(novic, axis = 1)

In [60]:
geih_vic_22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 919459 entries, 0 to 919458
Columns: 363 entries, DIRECTORIO to NO_VICTIMA
dtypes: category(1), float64(290), int64(48), object(24)
memory usage: 2.5+ GB


In [56]:
geih_vic_22["ES_VICTIMA"].value_counts()

ES_VICTIMA
1.0    123269
Name: count, dtype: int64

In [58]:
geih_vic_22["NO_VICTIMA"].value_counts()

NO_VICTIMA
1    796190
0    123269
Name: count, dtype: int64

## Estadísticas de validación

In [73]:
print(list(geih_vic_22.columns))

['DIRECTORIO', 'SECUENCIA_P', 'ORDEN', 'P6016', 'P3271', 'P6030S1', 'P6030S3', 'P6040', 'P6050', 'P6083', 'P6083S1', 'P6081', 'P6081S1', 'P2057', 'P2059', 'P2061', 'P6080', 'P6080S1', 'P6080S1A1', 'P6070', 'P6071', 'P6071S1', 'P3147S1', 'P3147S2', 'P3147S3', 'P3147S4', 'P3147S5', 'P3147S6', 'P3147S7', 'P3147S8', 'P3147S9', 'P3147S10', 'P3147S11', 'P3147S10A1', 'P6090', 'P6100', 'P6110', 'P6120', 'P1906S1', 'P1906S2', 'P1906S3', 'P1906S4', 'P1906S5', 'P1906S6', 'P1906S7', 'P1906S8', 'P6160', 'P6170', 'P3041', 'P3042', 'P3042S1', 'P3042S2', 'P3043', 'P3043S1', 'HOGAR', 'CLASE', 'P3038', 'P3039', 'AREA', 'MES', 'PERIODO', 'DPTO', 'PT', 'FEX_C18', 'LGB_NUMERICA', 'LGB_SECTORES', 'TRANS_NUMERICA', 'LGBT_NUMERICA', 'DISCAPACIDAD', 'DIFICULTAD', 'CAMPESINA', 'POB_MAY18', 'PER', 'REGIS', 'P6240', 'P6240S1', 'P6250', 'P6260', 'P6260S1', 'P6260S1A1', 'P6260S2', 'P6270', 'P6280', 'P6290', 'P6300', 'P6310', 'P6310S1', 'P6320', 'P6330', 'P6340', 'P6350', 'P6351', 'P6240S2', 'P3362S1', 'P3362S2', 'P

In [69]:
#prueba para el primer trimestre 2022
print("población nacional:", (pd.to_numeric(geih_vic_22[geih_vic_22["MES"]==1]['FEX_C18']).sum() + 
 pd.to_numeric(geih_vic_22[geih_vic_22["MES"]==2]['FEX_C18']).sum() + 
 pd.to_numeric(geih_vic_22[geih_vic_22["MES"]==3]['FEX_C18']).sum())/3 )
#pd.to_numeric(Enero_22[Enero_22['P6240']==1]['FEX_C18']).mean()

# prueba población campesina
print("población campesina:", 
((pd.to_numeric(geih_vic_22[(geih_vic_22["MES"]==1) & (geih_vic_22["P2057"]==1) & (geih_vic_22["P2061"]==1)]['FEX_C18']).sum()) +
(pd.to_numeric(geih_vic_22[(geih_vic_22["MES"]==2) & (geih_vic_22["P2057"]==1) & (geih_vic_22["P2061"]==1)]['FEX_C18']).sum()) +
(pd.to_numeric(geih_vic_22[(geih_vic_22["MES"]==3) & (geih_vic_22["P2057"]==1) & (geih_vic_22["P2061"]==1)]['FEX_C18']).sum()) 
)/ 3 )

población nacional: 50297750.33333716
población campesina: 14460983.610993758


In [83]:
# prueba población campesina con variable campesina
print("población campesina:", 
((pd.to_numeric(geih_vic_22[(geih_vic_22["MES"]==10) & (geih_vic_22["CAMPESINA"]==1)]['FEX_C18']).sum()) +
(pd.to_numeric(geih_vic_22[(geih_vic_22["MES"]==11) & (geih_vic_22["CAMPESINA"]==1)]['FEX_C18']).sum()) +
(pd.to_numeric(geih_vic_22[(geih_vic_22["MES"]==12) & (geih_vic_22["CAMPESINA"]==1)]['FEX_C18']).sum()) 
)/ 3 )

población campesina: 15545069.667519398


In [75]:
geih_vic_22['CAMPESINA'].value_counts()

CAMPESINA
0    738952
1    172923
9      7584
Name: count, dtype: int64

## Exportar base datos

In [92]:
"E:/GEIH/2022/salidas/geih_vic_22.csv"

'E:/GEIH/2022/salidas/'

In [96]:
# geih 2022
geih_vic_22.to_csv(p2 + "geih_vic_22.csv", sep= ";", index= False)

In [ ]:
# renombrar variables
#geih_vic_22.rename(columns = {"P3271":"SEXO", "P6040":"EDAD"}, inplace = True)